# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [2]:
pip install imageio


In [3]:
pip install matplotlib


In [4]:
#import numpy as np
#import os
#from scipy.misc import imread, imresize
#import datetime
#import os
import numpy as np
import os
from imageio import imread  # or use matplotlib.image.imread
import datetime
import os


We set the random seed so that the results don't vary drastically.

In [5]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In [6]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [7]:
train_doc = np.random.permutation(open('/content/drive/MyDrive/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/drive/MyDrive/Project_data/val.csv').readlines())
batch_size = 19

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [8]:
from skimage.io import imread
import numpy as np
import os
from skimage.transform import resize

def generator(source_path, folder_list, batch_size):
    print('Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [i for i in range(30)]  # Example: Creating a list of image numbers (assuming 30 images per video)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size  # calculate the number of batches
        for batch in range(num_batches):  # iterate over the number of batches
            batch_data = np.zeros((batch_size, 30, 120, 120, 3))  # initialize batch_data
            batch_labels = np.zeros((batch_size, 5))  # initialize batch_labels
            for folder in range(batch_size):  # iterate over the batch_size
                folder_path = os.path.join(source_path, t[folder + (batch*batch_size)].split(';')[0])
                imgs = os.listdir(folder_path)
                # Ensure there are enough images, otherwise skip or handle differently
                if len(imgs) < len(img_idx):
                    print(f"Not enough images in {folder_path}. Expected: {len(img_idx)}, Found: {len(imgs)}")
                    continue
                for idx, item in enumerate(img_idx):  # iterate over the frames/images of a folder to read them in
                    try:
                        image_path = os.path.join(folder_path, imgs[item])
                        image = imread(image_path).astype(np.float32)
                        # Crop and resize the image here if necessary
                        image_resized = resize(image, (120, 120), anti_aliasing=True)
                        # Normalize and feed in the image
                        for c in range(3):  # Iterate over channels
                            batch_data[folder, idx, :, :, c] = (image_resized[:, :, c] - np.mean(image_resized[:, :, c])) / np.std(image_resized[:, :, c])
                    except Exception as e:
                        print(f"Error loading image: {image_path}, error: {e}")
                        # Handle the error or use a placeholder image

                batch_labels[folder, int(t[folder + (batch*batch_size)].split(';')[2])] = 1  # set the corresponding label
            yield batch_data, batch_labels  # yield the batch_data and the batch_labels

        # Handle the last batch if it's smaller than batch_size
        rem = len(folder_list) % batch_size
        if rem != 0:
            batch_data = np.zeros((rem, 30, 120, 120, 3))
            batch_labels = np.zeros((rem, 5))
            for folder in range(rem):
                folder_path = os.path.join(source_path, t[folder + (num_batches*batch_size)].split(';')[0])
                imgs = os.listdir(folder_path)
                if len(imgs) < len(img_idx):
                    print(f"Not enough images in {folder_path}. Expected: {len(img_idx)}, Found: {len(imgs)}")
                    continue
                for idx, item in enumerate(img_idx):
                    try:
                        image_path = os.path.join(folder_path, imgs[item])
                        image = imread(image_path).astype(np.float32)
                        image_resized = resize(image, (120, 120), anti_aliasing=True)
                        for c in range(3):
                            batch_data[folder, idx, :, :, c] = (image_resized[:, :, c] - np.mean(image_resized[:, :, c])) / np.std(image_resized[:, :, c])
                    except Exception as e:
                        print(f"Error loading image: {image_path}, error: {e}")
                        # Handle the error or use a placeholder image

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].split(';')[2])] = 1
            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [9]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/drive/MyDrive/Project_data/train'
val_path = '/content/drive/MyDrive/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, BatchNormalization, Activation
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau



In [11]:
pip install keras


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, BatchNormalization, Activation
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers



In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, BatchNormalization, Activation
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

def conv3d_model(input_shape=(30, 120, 120, 3), num_classes=5):
    model = Sequential()

    # First convolutional layer
    model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=input_shape, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    # Second convolutional layer
    model.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    # Third convolutional layer
    model.add(Conv3D(128, kernel_size=(3, 3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling3D(pool_size=(2, 2, 2)))

    # Flatten layer to feed into GRU
    model.add(TimeDistributed(Flatten()))

    # GRU layer
    model.add(GRU(128, return_sequences=False))

    # Fully connected layers
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Define model
model = conv3d_model()

# Compile model
optimizer = optimizers.Adam(learning_rate=0.001)  # Updated from 'lr' to 'learning_rate'
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 120, 120, 32   2624      
                             )                                   
                                                                 
 activation (Activation)     (None, 30, 120, 120, 32   0         
                             )                                   
                                                                 
 batch_normalization (Batch  (None, 30, 120, 120, 32   128       
 Normalization)              )                                   
                                                                 
 max_pooling3d (MaxPooling3  (None, 15, 60, 60, 32)    0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 15, 60, 60, 64)    5

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [15]:
from tensorflow.keras.optimizers import Adam
optimiser = Adam(learning_rate=0.001)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 120, 120, 32   2624      
                             )                                   
                                                                 
 activation (Activation)     (None, 30, 120, 120, 32   0         
                             )                                   
                                                                 
 batch_normalization (Batch  (None, 30, 120, 120, 32   128       
 Normalization)              )                                   
                                                                 
 max_pooling3d (MaxPooling3  (None, 15, 60, 60, 32)    0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 15, 60, 60, 64)    5

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [16]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [17]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import datetime

# Assuming curr_dt_time is defined as the current datetime for naming the model directory
curr_dt_time = datetime.datetime.now()

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# Define ReduceLROnPlateau
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

callbacks_list = [checkpoint, LR]


The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [18]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [19]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /content/drive/MyDrive/Project_data/train ; batch size = 19
Epoch 1/20
35/35 [==============================] - ETA: 0s - loss: 1.5843 - categorical_accuracy: 0.2821  Source path =  /content/drive/MyDrive/Project_data/val ; batch size = 19

Epoch 1: saving model to model_init_2024-03-3116_24_32.781725/model-00001-1.58434-0.28205-1.54521-0.27000.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


35/35 [==============================] - 15179s 446s/step - loss: 1.5843 - categorical_accuracy: 0.2821 - val_loss: 1.5452 - val_categorical_accuracy: 0.2700 - lr: 0.0010
Epoch 2/20
35/35 [==============================] - ETA: 0s - loss: 1.3931 - categorical_accuracy: 0.3846
Epoch 2: saving model to model_init_2024-03-3116_24_32.781725/model-00002-1.39313-0.38462-1.62035-0.31000.h5
35/35 [==============================] - 225s 7s/step - loss: 1.3931 - categorical_accuracy: 0.3846 - val_loss: 1.6204 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 3/20
35/35 [==============================] - ETA: 0s - loss: 1.2890 - categorical_accuracy: 0.4932
Epoch 3: saving model to model_init_2024-03-3116_24_32.781725/model-00003-1.28896-0.49321-1.94364-0.27000.h5
35/35 [==============================] - 214s 6s/step - loss: 1.2890 - categorical_accuracy: 0.4932 - val_loss: 1.9436 - val_categorical_accuracy: 0.2700 - lr: 0.0010
Epoch 4/20
35/35 [==============================] - ETA: 0s - los